<a href="https://colab.research.google.com/github/vieduy/CS114.K21/blob/master/German_Traffic_Sign_Benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd drive/My\ Drive

/content/drive/My Drive


In [0]:
cd '..'

/content


In [0]:
!ls

'My Drive'


In [2]:
from config import dogs_vs_cats_config as config
print(config.NUM_TEST_IMAGES)

0.1


In [0]:
import matplotlib

# import the necessary packages
from scripts.preprocessing.imagetoarraypreprocessor import ImageToArrayPreprocessor
from scripts.preprocessing.simplepreprocessor import SimplePreprocessor
from scripts.preprocessing.patchpreprocessor import PatchPreprocessor
from scripts.preprocessing.meanpreprocessor import MeanPreprocessor
from scripts.callbacks.trainingmonitor import TrainingMonitor
from scripts.io.hdf5datasetgenerator import HDF5DatasetGenerator
from scripts.preprocessing.aspectawarepreprocessor import AspectAwarePreprocessor
from scripts.cnn.alexnet import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from scripts.io.simpledatasetloader import SimpleDatasetLoader
import json
import os
from imutils import paths
import random
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
aap = AspectAwarePreprocessor(64, 64)
iap = ImageToArrayPreprocessor()

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [0]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# List available datasets.
!kaggle datasets list

In [0]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

In [0]:
!unzip gtsrb-german-traffic-sign.zip

In [0]:
imagePaths = list(paths.list_images('../../Train'))
random.shuffle(imagePaths)
length = len(imagePaths)
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=50)
data = data.astype("float") / 255.0
print(labels)

[INFO] processed 50/39209
[INFO] processed 100/39209
[INFO] processed 150/39209
[INFO] processed 200/39209
[INFO] processed 250/39209
[INFO] processed 300/39209
[INFO] processed 350/39209
[INFO] processed 400/39209
[INFO] processed 450/39209
[INFO] processed 500/39209
[INFO] processed 550/39209
[INFO] processed 600/39209
[INFO] processed 650/39209
[INFO] processed 700/39209
[INFO] processed 750/39209
[INFO] processed 800/39209
[INFO] processed 850/39209
[INFO] processed 900/39209
[INFO] processed 950/39209
[INFO] processed 1000/39209
[INFO] processed 1050/39209
[INFO] processed 1100/39209
[INFO] processed 1150/39209
[INFO] processed 1200/39209
[INFO] processed 1250/39209
[INFO] processed 1300/39209
[INFO] processed 1350/39209
[INFO] processed 1400/39209
[INFO] processed 1450/39209
[INFO] processed 1500/39209
[INFO] processed 1550/39209
[INFO] processed 1600/39209
[INFO] processed 1650/39209
[INFO] processed 1700/39209
[INFO] processed 1750/39209
[INFO] processed 1800/39209
[INFO] proce

In [0]:
from sklearn.preprocessing import LabelBinarizer
import h5py

db = h5py.File('../../data.hdf5', 'r')
trainX = db["images"][:31368]
trainY = db["labels"][:31368]
testX = db["images"][31368:]
testY = db["labels"][31368:]

# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [0]:
# initialize the optimizer and model
from scripts.cnn.alexnet import AlexNet
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = AlexNet.build(width=64, height=64, depth=3,
  classes=43, reg=0.0002)
model.compile(loss="categorical_crossentropy", optimizer=opt,
  metrics=["accuracy"])

[INFO] compiling model...


In [0]:
print("[INFO] training network...")
model.fit(trainX, trainY, validation_data=(testX, testY),
  batch_size=64, epochs=10, verbose=1)

[INFO] training network...
Epoch 1/10
491/491 [==============================] - 28s 56ms/step - loss: 4.8025 - accuracy: 0.2350 - val_loss: 3.9202 - val_accuracy: 0.3497
Epoch 2/10
491/491 [==============================] - 27s 54ms/step - loss: 2.4851 - accuracy: 0.6367 - val_loss: 2.2680 - val_accuracy: 0.6735
Epoch 3/10
491/491 [==============================] - 26s 54ms/step - loss: 1.9019 - accuracy: 0.7761 - val_loss: 1.7842 - val_accuracy: 0.7786
Epoch 4/10
491/491 [==============================] - 27s 54ms/step - loss: 1.6889 - accuracy: 0.8265 - val_loss: 2.0442 - val_accuracy: 0.7310
Epoch 5/10
491/491 [==============================] - 27s 54ms/step - loss: 1.5937 - accuracy: 0.8538 - val_loss: 1.6791 - val_accuracy: 0.8112
Epoch 6/10
491/491 [==============================] - 27s 54ms/step - loss: 1.3432 - accuracy: 0.8900 - val_loss: 1.6496 - val_accuracy: 0.8384
Epoch 7/10
491/491 [==============================] - 27s 54ms/step - loss: 1.3186 - accuracy: 0.9014 - val_l

In [0]:
model.save('../../alexnet.model', overwrite=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../../alexnet.model/assets


In [6]:
from tensorflow.keras.models import load_model

print("[INFO] loading model...")
model = load_model('../../alexnet.model')

[INFO] loading model...


In [17]:
from os.path import basename, splitext

imagePaths = list((paths.list_images('../../Test')))
new_list = [splitext(basename(x))[0] for x in imagePaths]
fin_list = list(zip(imagePaths, new_list))
fin_list = [x[0] for x in sorted(fin_list,key=lambda x: int(x[1]))]
print(fin_list[0])
length = len(imagePaths)
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
data = sdl.load_test(fin_list, verbose=50, length=length)
data = data.astype("float") / 255.0
print(len(data))

../../Test/00000.png
[INFO] processed 50/12630
[INFO] processed 100/12630
[INFO] processed 150/12630
[INFO] processed 200/12630
[INFO] processed 250/12630
[INFO] processed 300/12630
[INFO] processed 350/12630
[INFO] processed 400/12630
[INFO] processed 450/12630
[INFO] processed 500/12630
[INFO] processed 550/12630
[INFO] processed 600/12630
[INFO] processed 650/12630
[INFO] processed 700/12630
[INFO] processed 750/12630
[INFO] processed 800/12630
[INFO] processed 850/12630
[INFO] processed 900/12630
[INFO] processed 950/12630
[INFO] processed 1000/12630
[INFO] processed 1050/12630
[INFO] processed 1100/12630
[INFO] processed 1150/12630
[INFO] processed 1200/12630
[INFO] processed 1250/12630
[INFO] processed 1300/12630
[INFO] processed 1350/12630
[INFO] processed 1400/12630
[INFO] processed 1450/12630
[INFO] processed 1500/12630
[INFO] processed 1550/12630
[INFO] processed 1600/12630
[INFO] processed 1650/12630
[INFO] processed 1700/12630
[INFO] processed 1750/12630
[INFO] processed 18

In [18]:
from sklearn.preprocessing import LabelBinarizer
import h5py

db = h5py.File('../../test.hdf5', 'r')
testImages = db["images"]
predictions = model.predict(testImages ,batch_size=64)
pred = predictions.argmax(axis=1)

In [19]:
import pandas as pd

test_df = pd.read_csv('Test.csv')
labels = test_df['ClassId'].values
print(labels[1000], pred[1000])

17 17


In [20]:
from sklearn.metrics import classification_report

print(classification_report(labels,
  predictions.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.86      0.32      0.46        60
           1       0.76      0.98      0.85       720
           2       0.93      0.89      0.91       750
           3       0.72      0.71      0.72       450
           4       0.94      0.75      0.84       660
           5       0.68      0.78      0.73       630
           6       0.93      0.50      0.65       150
           7       0.73      0.83      0.78       450
           8       0.86      0.82      0.84       450
           9       0.97      0.85      0.91       480
          10       0.85      0.99      0.91       660
          11       0.92      0.91      0.92       420
          12       0.87      0.95      0.91       690
          13       0.97      0.99      0.98       720
          14       0.99      0.98      0.99       270
          15       0.99      0.91      0.95       210
          16       1.00      0.95      0.97       150
          17       1.00    

In [21]:
 from sklearn.metrics import accuracy_score

 print(accuracy_score(labels, pred))

0.8494853523357087
